# Parse Tikz Code to Dagitty DAG:

In [2]:
import re
def parse_nodes_to_DAG(nodes, exposure:list, outcome:list, adjusted:list):
    """
    \node (v18) at (0.407,-2.84) {# Meals/D};
    ->
    #-Meals/D [pos="0.407,-2.84"]
    """
    result_text = []
    nid_mappings = {}
    for i,n in enumerate(nodes):
        if len(n) > 0:
            node_data = re.findall( r'\((.+?)\)', n)
            (nid, (posx, posy)) = node_data[0], node_data[1].split(',')
            node_text = re.findall( r'\{(.+?)\}', n)[0]#.replace(' ','-')
            is_e = node_text in exposure
            is_o = node_text in outcome
            is_a = node_text in adjusted
            attr = 'exposure,' if is_e else 'outcome,' if is_o else 'adjusted,' if is_a else ''
            posy = f'-{posy}' if '-' not in posy else posy[1:]
            nid_mappings[nid] = f'"{node_text}"'
            result_text += [f'"{node_text}" [{attr}pos="{posx},{posy}"]']    #Coach [pos="-4.392,-7.906"]
    return result_text, nid_mappings

def parse_edges_to_DAG(edges, nid_mappings):
    """
    \draw [->] (v32) edge (v31);
    ->
    TeamMotivation -> PreviousInjury
    """
    result_text = []
    for i,n in enumerate(edges):
        if len(n) > 0:
            (n1, n2) = re.findall( r'\((.+?)\)', n)
            result_text += [f'{nid_mappings[n1]} -> {nid_mappings[n2]}']
    return result_text

In [3]:
# % This code uses the tikz package
# \begin{tikzpicture}
nodes = """\node (v0) at (-7.00,-8.65) {WarmUpExercises};
\node (v1) at (4.97,-8.61) {Injury};
\node (v2) at (-4.39,7.91) {Coach};
\node (v3) at (2.02,7.91) {Genetics};
\node (v4) at (-7.17,0.950) {TeamMotivation};
\node (v5) at (-3.64,1.24) {PreGameProprioception};
\node (v6) at (3.49,5.10) {ConnectiveTissueDisorder};
\node (v7) at (-4.29,-4.17) {PreviousInjury};
\node (v8) at (-1.16,-2.55) {ContactSport};
\node (v9) at (4.24,-1.86) {TissueWeakness};
\node (v10) at (-1.20,-8.49) {IntraGameProprioception};
\node (v11) at (-1.49,4.53) {FitnessLevel};
\node (v12) at (1.50,1.24) {NeuromuscularFatigue};
"""
edges = """\draw [->] (v2) edge (v11);
\draw [->] (v2) edge (v4);
\draw [->] (v3) edge (v11);
\draw [->] (v3) edge (v12);
\draw [->] (v3) edge (v6);
\draw [->] (v4) edge (v7);
\draw [->] (v4) edge (v0);
\draw [->] (v5) edge (v0);
\draw [->] (v6) edge (v9);
\draw [->] (v6) edge (v12);
\draw [->] (v8) edge (v10);
\draw [->] (v8) edge (v7);
\draw [->] (v9) edge (v1);
\draw [->] (v10) edge (v1);
\draw [->] (v11) edge (v5);
\draw [->] (v11) edge (v12);
\draw [->] (v12) edge (v10);
\draw [->] (v12) edge (v1);
\draw [->] (v0) edge (v10);"""
# \end{tikzpicture}

In [4]:
nodes = nodes.replace('\node ','').replace('\n','').split(';')
edges = edges.replace('\draw [->] ','').replace('\n','').split(';')
exposure = ['WarmUpExercises']
outcome = ['Injury']
adjusted = []

In [5]:
result_text, nid_mappings = parse_nodes_to_DAG(nodes, exposure, outcome, adjusted)
result_text += parse_edges_to_DAG(edges, nid_mappings)
result_text = ['dag {'] + result_text + ['}']

for t in result_text:
    print(t)

dag {
"WarmUpExercises" [exposure,pos="-7.00,8.65"]
"Injury" [outcome,pos="4.97,8.61"]
"Coach" [pos="-4.39,-7.91"]
"Genetics" [pos="2.02,-7.91"]
"TeamMotivation" [pos="-7.17,-0.950"]
"PreGameProprioception" [pos="-3.64,-1.24"]
"ConnectiveTissueDisorder" [pos="3.49,-5.10"]
"PreviousInjury" [pos="-4.29,4.17"]
"ContactSport" [pos="-1.16,2.55"]
"TissueWeakness" [pos="4.24,1.86"]
"IntraGameProprioception" [pos="-1.20,8.49"]
"FitnessLevel" [pos="-1.49,-4.53"]
"NeuromuscularFatigue" [pos="1.50,-1.24"]
"Coach" -> "FitnessLevel"
"Coach" -> "TeamMotivation"
"Genetics" -> "FitnessLevel"
"Genetics" -> "NeuromuscularFatigue"
"Genetics" -> "ConnectiveTissueDisorder"
"TeamMotivation" -> "PreviousInjury"
"TeamMotivation" -> "WarmUpExercises"
"PreGameProprioception" -> "WarmUpExercises"
"ConnectiveTissueDisorder" -> "TissueWeakness"
"ConnectiveTissueDisorder" -> "NeuromuscularFatigue"
"ContactSport" -> "IntraGameProprioception"
"ContactSport" -> "PreviousInjury"
"TissueWeakness" -> "Injury"
"IntraGameP